In [4]:
# Cloud Mask 2.0 

import ee
import yaml
import time
import os
import json
from argparse import ArgumentParser
from utils import clipToROI, exportImageCollectionToGCS, exportImageToGCS, sentinel2CloudScore, calcCloudCoverage
from utils import GEETaskManager

from gevent.fileobject import FileObjectThread

# Polygon Import from Misha ROI List

import pandas as pd

# Functions for Active Run of Cloud Mask 2.0 

from download_sen12 import *

In [5]:
config_file = "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/config.yml"

In [6]:
stream = open(config_file, 'r') 

In [7]:
config = yaml.load(stream)

<ipython-input-7-8d98dc56653f>:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(stream)


In [8]:
config["data_list"][0]

{'name': ['summer', '1886'],
 'start_date': '2017-06-01',
 'end_date': '2017-08-31',
 'geometry': 'point',
 'size': 20000,
 'resolution': 10,
 'sort_by': 'name',
 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI',
 'sensors': [0, 1, 2]}

# Initialize EE

In [9]:
ee.Initialize()

# Creating New Feature Collection To Use with CM_v2

In [10]:
def import_aois(csv_loc):    

    df_labels = pd.read_csv(csv_loc)
    df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

    polygons = []
    for polygon in df_labels["polygon"]:
        polygons.append(json.loads(polygon)["coordinates"])

    return polygons

polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

feature_id = 0 
features = []
for poly in polygons[0:3]:
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)

fc = ee.FeatureCollection(features)

In [11]:
fc

In [12]:
task_queue = GEETaskManager(n_workers=config['max_tasks'], max_retry=config['max_retry'], wake_on_task=True, log_file=config['log_file'], process_timeout=config['task_timeout'])
task_queue.register_monitor(monitor_tasks)

if os.path.exists('task_log.json'):
    task_log = load_task_log(filename='task_log.json')
    task_queue.set_task_log(task_log)

for data_list in config['data_list'][0:1]:
    for sensor_idx in data_list['sensors']:
        sensor = config['sensors'][sensor_idx]
        print(sensor)
        tasks = process_datasource(task_queue, data_list, sensor, config['export_to'], config['export_dest'], feature_list = fc)

print("Waiting for completion...")
task_queue.wait_till_done()

{'name': 'COPERNICUS/S2', 'prefix': 'S2_CloudFree', 'type': 'opt', 'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'], 'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]}
3 features have been loaded
Queued Task S2_CloudFree_summer_1886_1
Another exception occured
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers


Traceback (most recent call last):
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 140, in _worker
    self.worker(args)
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 71, in _default_worker
    g_task = task_def['action'](**task_def['kwargs'])
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/download_sen12.py", line 108, in export_single_feature
    if sensor['type'].lower() == "opt":
NameError: name 'sensor' is not defined
Traceback (most recent call last):
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 140, in _worker
    self.worker(args)
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 71, in _default_worker
    g_task = task

Queued Task S2_CloudFree_summer_1886_2
{'name': 'COPERNICUS/S2', 'type': 'opt', 'prefix': 'S2_20_60_Cloud', 'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'], 'filters_after': [{'CLOUDY_PERCENTAGE': {'gte': 20}}, {'CLOUDY_PERCENTAGE': {'lte': 60}}]}
Another exception occured
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers
3 features have been loaded
Queued Task S2_20_60_Cloud_summer_1886_1
Another exception occured
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers
Queued Task S2_20_60_Cloud_summer_1886_2
{'name': 'COPERNICUS/S1_GRD', 'type': 'sar', 'prefix': 'S1', 'bands': ['VV', 'VH'], 'filters_before': [{'instrumentMode': {'eq': 'IW'}}, {'transmitterReceiverPolarisation': {'listContains': 'VV'}}, {'transmitterReceiverPolarisation': {'listContains': 'VH'}}]}
Another exception occured
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers
3 fea

Traceback (most recent call last):
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 140, in _worker
    self.worker(args)
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 71, in _default_worker
    g_task = task_def['action'](**task_def['kwargs'])
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/download_sen12.py", line 108, in export_single_feature
    if sensor['type'].lower() == "opt":
NameError: name 'sensor' is not defined
Traceback (most recent call last):
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 140, in _worker
    self.worker(args)
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 71, in _default_worker
    g_task = task

Queued Task S1_summer_1886_1
Another exception occured
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers
Queued Task S1_summer_1886_2
Waiting for completion...


Traceback (most recent call last):
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 140, in _worker
    self.worker(args)
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 71, in _default_worker
    g_task = task_def['action'](**task_def['kwargs'])
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/download_sen12.py", line 108, in export_single_feature
    if sensor['type'].lower() == "opt":
NameError: name 'sensor' is not defined


AttributeError: 'NoneType' object has no attribute 'rawlink'

Another exception occured
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers


Traceback (most recent call last):
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 140, in _worker
    self.worker(args)
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/utils/gee_task_manager.py", line 71, in _default_worker
    g_task = task_def['action'](**task_def['kwargs'])
  File "/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader/download_sen12.py", line 108, in export_single_feature
    if sensor['type'].lower() == "opt":
NameError: name 'sensor' is not defined


In [46]:
task_queue = GEETaskManager(n_workers=config['max_tasks'], max_retry=config['max_retry'], wake_on_task=True, log_file=config['log_file'], process_timeout=config['task_timeout'])
task_queue.register_monitor(monitor_tasks)

if os.path.exists('task_log.json'):
    task_log = load_task_log(filename='task_log.json')
    task_queue.set_task_log(task_log)

for data_list in config['data_list']:
    for sensor_idx in data_list['sensors']:
        sensor = config['sensors'][sensor_idx]
        tasks = process_datasource(task_queue, data_list, sensor, config['export_to'], config['export_dest'])

print("Waiting for completion...")
task_queue.wait_till_done()

EEException: Collection.loadTable: Collection asset 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI' not found.

In [19]:
!pwd

/Users/purgatorid/Documents/GitHub/canopy-gis/google-earth-explore/GEE_DataDownloader
